### 5.3 Programming Task: Digit recognition using CNNs

In [46]:
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms
from torchinfo import summary


%matplotlib inline

In [47]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(20 * 12 * 12, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 20 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [48]:
net = ConvNet()
print(net)

ConvNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2880, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


In [49]:
batch_size = 64
learning_rate = 0.001
epochs = 3
device = torch.device('cpu') 


In [50]:
# Load the MNIST data set
transform = transforms.Compose([
    transforms.ToTensor()
])

train_set = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST('./data', train=False, transform=transform)

train_loader = Data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = Data.DataLoader(test_set, batch_size=batch_size, shuffle=False)



In [51]:
net.to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [52]:
net.train()
for epoch in range(epochs):
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        output = net(data)
        
        loss = criterion(output, target)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.2057
Epoch 2, Loss: 0.0635
Epoch 3, Loss: 0.0434


In [53]:
net.eval()
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = net(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

print(f'\nTest Accuracy: {100. * correct / len(test_loader.dataset)}%')


Test Accuracy: 98.61%


In [54]:
summary(net, input_size=(1, 1, 28, 28), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
ConvNet                                  [1, 10]                   --
├─Conv2d: 1-1                            [1, 20, 24, 24]           520
├─MaxPool2d: 1-2                         [1, 20, 12, 12]           --
├─Linear: 1-3                            [1, 100]                  288,100
├─Linear: 1-4                            [1, 10]                   1,010
Total params: 289,630
Trainable params: 289,630
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.59
Input size (MB): 0.00
Forward/backward pass size (MB): 0.09
Params size (MB): 1.16
Estimated Total Size (MB): 1.25